In [1]:
import numpy as np
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
import heapq

import csv


In [2]:
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora

In [3]:
import math

In [4]:

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
Question="collateral"
file = open('Obligations of the mortgager.pdf','rb')

In [ ]:
file = open('Obligations of the mortgager.pdf','rb')
PDF_read = PdfFileReader(file)
txt=""
for i in range(PDF_read.numPages):
    page_temp = PDF_read.getPage(i)
    page_String= page_temp.extractText()
    txt=txt+ page_String
    


In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

   
tokens = nltk.sent_tokenize(txt) 
tokens = tokens
tfidfvectoriser=TfidfVectorizer()

In [74]:
article_df= pd.read_csv("cl_art_ten_dim384_clus35.csv",index_col=[0])
centroid_df= pd.read_csv("centroids_dim384_clus35.csv", index_col=[0])
sentence_df= pd.read_csv("setences_text_384.csv",index_col=[0])

In [7]:
query_df=pd.read_csv("query.csv")

In [8]:
query_df.columns=['Dimension', 'Value']

In [9]:
query_tensor=query_df['Value'].to_numpy()


In [ ]:

def clean_sentence(sentence, stopwords=False):
    word_tokens = word_tokenize(sentence)
 
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        
    filtered_sentence = ""
        
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence=filtered_sentence+w

    return filtered_sentence

In [ ]:
def get_cleaned_sentences(tokens, stopwords=False):

    cleaned_sentences = []
    for line in tokens:
      cleaned = clean_sentence(line, stopwords)
      cleaned_sentences.append(cleaned)

    return cleaned_sentences

In [ ]:
   
def cleanall(tokens):
                cleaned_sentences = get_cleaned_sentences(tokens, stopwords=True)
                cleaned_sentences_with_stopwords = get_cleaned_sentences(tokens, stopwords=False)
                # print(cleaned_sentences)
                # print(cleaned_sentences_with_stopwords)
                return [cleaned_sentences,cleaned_sentences_with_stopwords]

In [ ]:
cleaned_sentences, cleaned_sentences_with_stopwords=cleanall(tokens)
sentences = cleaned_sentences
sentence_words = [[word for word in document.split()]
                    for document in sentences]
dictionary = corpora.Dictionary(sentence_words)
    # for key, value in dictionary.items():
    #     print(key, ' : ', value)
corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent, embedding in zip(sentences, corpus):
    print(sent)
    print(embedding)

In [10]:
   
def TFIDF(cleaned_sentences):
                tfidfvectoriser.fit(cleaned_sentences)
                tfidf_vectors=tfidfvectoriser.transform(cleaned_sentences)
                return tfidf_vectors

In [11]:
    
def TFIDF_Q(question_to_be_cleaned):
                tfidf_vectors=tfidfvectoriser.transform([question_to_be_cleaned])
                return tfidf_vectors

In [12]:

def doall(tokens):
                cleaned_sentences, cleaned_sentences_with_stopwords = cleanall(tokens)
                tfidf = TFIDF(cleaned_sentences)
                return [cleaned_sentences,cleaned_sentences_with_stopwords,tfidf]

In [13]:

    #cosine similarity
def Cosine(question_vector, sentence_vector):
        dot_product = np.dot(question_vector, sentence_vector.T)
        denominator = (np.linalg.norm(question_vector) * np.linalg.norm(sentence_vector))
        return dot_product/denominator

In [42]:

def Euclidean(question_vector, sentence_vector):
                vec1 = question_vector.copy()
                vec2 = sentence_vector.copy()
                if len(vec1)<len(vec2): vec1,vec2 = vec2,vec1
                vec2 = np.resize(vec2,(vec1.shape[0]))
                return np.linalg.norm(vec1-vec2)

In [15]:
def Theta(question_vector, sentence_vector):
        return np.arccos(Cosine(question_vector, sentence_vector)) + np.radians(10)

In [16]:
def Triangle(question_vector, sentence_vector):
        theta = np.radians(Theta(question_vector, sentence_vector))
        return ((np.linalg.norm(question_vector) * np.linalg.norm(sentence_vector)) * np.sin(theta))/2

In [17]:
def Magnitude_Difference(vec1, vec2):
        return abs((np.linalg.norm(vec1) - np.linalg.norm(vec2)))

In [18]:
def Sector(question_vector, sentence_vector):
        ED = Euclidean(question_vector, sentence_vector)
        MD = Magnitude_Difference(question_vector, sentence_vector)
        theta = Theta(question_vector, sentence_vector)
        return math.pi * (ED + MD)**2 * theta/360

In [19]:
 
def answer(question_vector, sentence_vector, method):
                if method==1: return Euclidean(question_vector,sentence_vector)
                else: return Cosine(question_vector,sentence_vector)

In [27]:
def TSSS_answer(question_vector, sentence_vector,method):
        if method==1: return Euclidean(question_vector, sentence_vector)
        elif method==2: return Cosine(question_vector, sentence_vector)
        else: return Triangle(question_vector, sentence_vector) * Sector(question_vector, sentence_vector)

In [31]:

def RetrieveAnswer(question_embedding, centroid_df,method):
  similarity_heap = []
  if method==1: 
    max_similarity = float('inf')
  else: max_similarity = -1
  index_similarity = -1

  for i in range (len(centroid_df)):
    similarity = TSSS_answer((question_embedding),(centroid_df.iloc[i]).to_numpy() , method).mean()
    if method==1:
      heapq.heappush(similarity_heap,(similarity,i))
    else:
      heapq.heappush(similarity_heap,(-similarity,i))
  return similarity_heap
  #for index, embedding in enumerate(centroid_df):  
    
   # similarity = answer((question_embedding).toarray(),(embedding).toarray() , method).mean()
    #if method==1:
    #  heapq.heappush(similarity_heap,(similarity,index))
    #else:
     # heapq.heappush(similarity_heap,(-similarity,index))
  #return similarity_heap


In [57]:
method = 3

#cleaned_sentences,cleaned_sentences_with_stopwords,tfidf_vectors = doall(tokens)

#question = clean_sentence(Question, stopwords=True)
#question_embedding = TFIDF_Q(question)
question_embedding= query_tensor

similarity_heap = RetrieveAnswer(question_embedding , centroid_df ,method)
#print("Question: ", Question)
cluster_list=[]
number_of_sentences_to_print = 5
while number_of_sentences_to_print>0 and len(similarity_heap)>0:
  x = similarity_heap.pop(1)
  #print(cleaned_sentences_with_stopwords[x[1]])
  cluster_list.append(x[1])



  number_of_sentences_to_print-=1

In [58]:
cluster_list

[29, 16, 19, 0, 12]

In [70]:
search_space = article_df[article_df['cluster'].isin(cluster_list)]

In [71]:
search_space.drop(['cluster','article'],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [75]:
sentence_df.head()

,sentence
0,We use certain terms in the mortgage deed and ...
1,Here we explain what we mean by it.
2,"Obvion: Obvion N.V., the lender that provides ..."
3,"In these terms and conditions, “you” always re..."
4,Is that more people?


In [73]:
similarity_heap_answer = RetrieveAnswer(question_embedding , search_space ,method=3)

In [76]:
number_of_sentences_to_print = 5
while number_of_sentences_to_print>0 and len(similarity_heap_answer)>0:
  x = similarity_heap.pop(1)
  print(sentence_df.iloc[x[1]])
  



  number_of_sentences_to_print-=1

sentence    Mortgage lender: The person or persons who giv...
Name: 6, dtype: object
sentence    This means, among other things: Obvion is allo...
Name: 15, dtype: object
sentence    Obvion has the Obvion Residential Mortgage, th...
Name: 18, dtype: object
sentence    For example the annuity mortgage.
Name: 20, dtype: object
sentence    Deed of loan:
Name: 22, dtype: object


In [52]:
centroid_df.rename(columns= {1: 'Cluster'},inplace=True)

In [38]:
centroid_df.iloc[0].to_numpy

<bound method IndexOpsMixin.to_numpy of 0     -1.465602
1     -0.363500
2      1.058877
3     -0.613788
4      1.409080
         ...   
379   -1.209984
380   -0.435533
381   -0.914766
382   -0.131569
383   -0.724329
Name: 0, Length: 384, dtype: float64>

In [69]:
centroid_df.iloc[34]

0      0.259726
1      0.837227
2     -0.278507
3     -0.756798
4      0.709814
         ...   
379   -2.178623
380    1.010109
381    0.901775
382   -1.853763
383   -1.407049
Name: 34, Length: 384, dtype: float64